# Nordlys RESTful API

This code retrieves the documents from DBPedia using the Nordlys RESTful API.

#### Get the list of queries

In [73]:
with open('query_list.txt', 'r') as f:
    text = f.read()
queries = text.split("\n")
queries = [(query.split('\t')[0], query.split('\t')[1]) for query in queries]

##### Get a dictionary with queries and their list of document relevances

In [74]:
with open('query_relevance.txt', 'r') as f:
    text = f.read()
    
text.split('\n')[0].split('\t')[0]

query_dict = {}

for query_doc in text.split('\n'):
    l = query_doc.split('\t')    
    if l[0] not in query_dict.keys():
        query_dict[l[0]] = {l[2][9:-1]: l[-1]}
    else:    
        query_dict[l[0]][l[2][9:-1]] = l[-1]


##### Get the resulting documents for each query and make a .txt file with the resulting documents and their relevances.

In [72]:
import requests
import json
import codecs

nordlys_retrieval_dict = {}
models = ['lm', 'bm25', 'prms', 'mlm']

for model in models:
    for query in queries:
        request_url = 'http://api.nordlys.cc/er?q=' + query[1].replace(' ', '+') +'&1st_num_docs=20&model=' + model
        r = requests.get(request_url)
        r.json()
        nordlys_retrieval_dict[query[0]] = []
        for key in r.json()['results'].keys():
            doc_title = r.json()['results'][key]['entity'][9:-1]
            if doc_title in query_dict[query[0]].keys():
                doc_rel = query_dict[query[0]][doc_title] 
            else:
                doc_rel = 0
            nordlys_retrieval_dict[query[0]].append([doc_title,  doc_rel])
        nordlys_retrieval_dict
        with codecs.open("nordlys_retrievals_" + model + ".txt", "w", "utf-8") as file:
            file.write(str(nordlys_retrieval_dict))